# Required packages, files install, config & loading

In [0]:
!pip install -q gensim
!pip install --upgrade -q gspread==0.6.2
!pip install -U -q PyDrive

In [0]:
import os
import numpy as np
import csv
import keras
import gensim
import random
import scipy
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras import regularizers
from keras import backend as K
from keras.optimizers import RMSprop
from keras.layers.core import Dense , Dropout , Activation, Flatten, Lambda
from keras.layers import Input, Embedding, LSTM, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, BatchNormalization, regularizers, Concatenate
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import EarlyStopping, ModelCheckpoint
from gensim.models import KeyedVectors 
#google drive libs
from oauth2client.client import GoogleCredentials
from google.colab import auth,files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread

In [0]:
#@title Authenticate Python Google Drive && Google Spread Sheet
auth.authenticate_user()
credential = GoogleCredentials.get_application_default()
#iniate Google Spread Sheet
gc = gspread.authorize(credential)
#iniate Google Drive
gauth = GoogleAuth()
gauth.credentials = credential
drive = GoogleDrive(gauth)


# Choose a local directory to store the data.
local_download_path = os.path.expanduser("~/data")
try:
  os.makedirs(local_download_path)
except: pass


#get the GoogleNews-vectors-negative300.bin.gz
file_id='0B7XkCwpI5KDYNlNUTTlSS21pQmM'
f = drive.CreateFile({'id': file_id})
print('title: %s, id: %s' % (f['title'], f['id']))
fname = os.path.join(local_download_path, f['title'])
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': f['id']})
f_.GetContentFile(fname)
print('imported the file into  System OS Path')

title: GoogleNews-vectors-negative300.bin.gz, id: 0B7XkCwpI5KDYNlNUTTlSS21pQmM
downloading to /root/data/GoogleNews-vectors-negative300.bin.gz
imported the file into  System OS Path


# **Using Pre-built GlobVe to generate embedding**

In [0]:
#@title text processing utilities
import re
def loaddata_fromcsv(filename): 
    worksheet = gc.open(filename)  
    activesheet = worksheet.get_worksheet(0)
    reader = activesheet.get_all_values()  
    dataset = [ ( filter_string(row[1]) , row[0]  ) for row in reader ]
    return dataset

#add data into cells
def w2sheet(data, cells):
    for i in range(len(data)):
      cells[i].value = data[i]
    return cells  
# Load dictionary of matches to expand
def load_dict():
    return {
        'DSL': 'DIESEL',
        'ENG': 'ENGINE',
        'MPDE': 'Main Propulsion Diesel Engine',
        'STBD': 'No.1 Main Engine',
        'PORT': 'No.2 Main Engine',
        'MDG': 'Main Diesel Generator',
        'MN PRPLN': 'Main PROPULSION',
        'SSDG': 'Ship Service Diesel Engine',
        'CNTR MAIN ENGINE': 'Center Main Engine',
        'SS GEN DIESEL ENGINE': 'Ship Service Generator Diesel Engine',
        'ME': 'Main Engine',
        'PTO': 'Power Take Off',
        'SS': 'Ship Service',
        'CRP': 'Contra-Rotating Propeller',
        'FO': 'Fuel Oil',
        'F.O.': 'Fuel Oil',
        'LO': 'Lube Oil',
        'HT': 'High temperature',
        'HP': 'High Presure',
        'EDG': 'Emergy Diesel Engine',
        'FW': 'Fresh Water',
        'ANT.': 'Antenna',
        'VDR': 'Voyage Data Recorder',
        'ACS': 'Aviation Control Station',
        'RED': 'REDUCTION',
        'L. O.': 'Lube Oil',
        'L.O.': 'Lube Oil',
        'PME': 'No.2 Main Engine',
        'ENG': 'Engine',
        'MN ENG': 'Main Engine',
        'SME': 'No.1 Main engine',
        'MN': 'Main',
        'WTR': 'Water',
        'HYD': 'Hydraulic',
        'CIRC': 'Circuit',
        'TK': 'Tank',
        'AFT': 'After',
        'FWD': 'Forward',
        'LUBO': 'Lube Oil',
        'CFW': 'Central Fresh Water',
        'FILL': 'Filter',
        'MTR': 'Meter',
        'PTO': 'Power Take Off',
        'GRP': 'Glass Reinforced Plastic',
        'DET': 'Detector',
        'CRPP': 'Controllable Reversible Pitch Propeller',
        'OD': 'Oil Distribution',
        'RES': 'Resilient',
        'EDG': 'Emergency Diesel Generator',
        'JW': 'Jacket Water',
        'J.W.': 'Jacket Water',
        'ERC': 'Engine Room Control',
        'TURBOCHANGER':'turbo charger',
        'LINESHAFT': 'LINE SHAFT',
        'OD' : 'Oil Distribution',
        'CFW' : 'Central Fresh Water',
        'VLV': 'Valve',
        'TURBO CHARGERS': 'Turbochargers',
        'TURBOCHARGERS': 'Turbochargers',
        'TURBOS': 'Turbochargers'
    }
      
# Replace words in sentence and rejoin
def filter_string(text):
    # Load replacement dictionary
    wordDic = load_dict()
    
    # Compile regex from dictionary
    # Regex looks for a word boundary, followed by a matching group for each
    # dictionary key, and then either a space or the end of the string, since
    # looking for a word boundary after a key with spaces in it doesn't act
    # as desired. This should prevent matches from inside words from being replaced
    regex = re.compile(r"\b(%s)(?=\s|$)" % "|".join(map(re.escape, wordDic.keys())), re.IGNORECASE)
    
    # Substitute matches in string based on regex
    # Cast each match to uppercase and get the matching value from the dictionary
    return regex.sub(lambda m: wordDic.get(m.group().upper()), text)

In [0]:
#@title Get embedding matrix from 'GoogleNews-vectors-negative300' { output-height: 12 }
print ("define input dimemsions parameters")
# size of the word embeddings
embeddings_dim = 300

# vocabulary size
vocabulary_size = 3000

# percentage of the data used for model training
#percent = 1

# number of classes
# num_classes = 2


print ("Reading pre-trained word embeddings...")
embeddings = dict( )
embeddings = KeyedVectors.load_word2vec_format('/root/data/GoogleNews-vectors-negative300.bin.gz', binary=True )

print ("Reading text data for classification and building representations...")
train_data = loaddata_fromcsv('Training82')
test_data = loaddata_fromcsv('Testing82')
#random.shuffle( train_data )

train_texts = [ txt.lower() for ( txt, label ) in train_data ]
test_texts = [ txt.lower() for ( txt, label ) in test_data ]
train_labels = [ label for ( txt , label ) in train_data ]
test_labels = [ label for ( txt , label ) in test_data ]
#num_classes = len( set( train_labels + test_labels ) )

# maximum length of a sentence
max_sent_len = max([len(x.split(" ")) for x in train_texts])
# max_sent_len = 35

print ("tokenlize input")

tokenizer = Tokenizer(num_words=vocabulary_size,lower=True, split=" ")
tokenizer.fit_on_texts(train_texts)

train_sequences = sequence.pad_sequences( tokenizer.texts_to_sequences( train_texts ) , maxlen=max_sent_len )
test_sequences = sequence.pad_sequences( tokenizer.texts_to_sequences( test_texts ) , maxlen=max_sent_len )

# train_matrix = tokenizer.texts_to_matrix( train_texts )
# test_matrix = tokenizer.texts_to_matrix( test_texts )

embedding_weights = np.zeros((vocabulary_size,embeddings_dim))
for word,index in tokenizer.word_index.items():
   if index < vocabulary_size:
       try: embedding_weights[index,:] = embeddings[word]
       except: embedding_weights[index,:] = np.random.rand( 1 , embeddings_dim )
           
lb = LabelBinarizer()
lb.fit(train_labels + test_labels)
train_labels = lb.transform(train_labels)
test_labels =lb.transform(test_labels)
print(train_labels.shape)
print(test_labels.shape)
print("Classes that are considered in the problem : {0}".format(repr( lb.classes_ )))

define input dimemsions parameters
Reading pre-trained word embeddings...
Reading text data for classification and building representations...
tokenlize input
(1417, 240)
(195, 240)
Classes that are considered in the problem : array(['233111', '23311133', '233111361', '2331113C', '23311141',
       '233111412', '23311142', '23311152', '23311161', '233111B',
       '233111B2', '233111B3', '233111N', '233121', '23312111',
       '23312112', '23312113', '23312114', '23312115', '23312116',
       '23312117', '23312118', '23312119', '2331211A', '23312133',
       '2331213C', '23312141', '233121412', '23312142', '23312152',
       '23312161', '2331218', '233121B', '233121B2', '233121B3',
       '233121N', '233121N1', '241111', '2411119', '241111A1', '241111F',
       '241121', '2411219', '241121A1', '241121F', '242111', '242112',
       '242113', '2421132', '242121', '242122', '242123', '2421232',
       '2431112', '24311121', '243114', '24311411', '24311411A',
       '24311411C', '24311411D

In [0]:
print(train_sequences)
print(train_sequences.shape)
print(test_sequences.shape)
print(train_labels.shape)
print(test_labels.shape)
print(max_sent_len)

[[  0   0   0 ...   0   0 243]
 [  0   0   0 ...   2  30  42]
 [151   3   6 ... 194 178  29]
 ...
 [  0   0   0 ... 115  33  34]
 [  0   0   0 ...   1 242 153]
 [  0   0   0 ...   1 242 153]]
(1417, 25)
(195, 25)
(1417, 240)
(195, 240)
25


In [0]:
import pickle
from google.colab import files

encoder = {"tokenizer":tokenizer, "classes_": lb.classes_}
print(encoder["classes_"].shape)

with open('encoder.pickle', 'wb') as file:
  pickle.dump(encoder,file)
files.download('encoder.pickle')

(240,)


# CNN 2.0 Implementation
https://github.com/dennybritz/cnn-text-classification-tf

In [0]:

print(embedding_weights.shape)
#variable initialization 
nb_filter = 300
LR = 0.0001
region_sizes=[1,2,3,5]

input_shape=(max_sent_len, embeddings_dim)
conv_input_shape =(max_sent_len, embeddings_dim,1)

# create seperate model graph for parallel processing with different filter sizes
# apply 'same' padding so that ll produce o/p tensor of same size for concatination

inp = Input(shape=input_shape)
lamb_inp = Lambda(K.expand_dims, output_shape=conv_input_shape)(inp)
# 4 region size: (2,3,4,5)    
# nb_filter filters for each region size
# totally 4* nb_filter filters
convs = []
for i in range(len(region_sizes)):
    conv = Conv2D(nb_filter, (region_sizes[i],300),
                  strides=1,
                  padding='valid',
                  activation='relu',
                  input_shape=conv_input_shape)(lamb_inp)
    #I-max pooling
    pool = MaxPooling2D(pool_size=(max_sent_len - region_sizes[i] + 1,1))(conv)

    convs.append(pool)

if len(region_sizes) > 1:
    # cancat all paralle output
    out = Concatenate()(convs)
else:
  out = convs
conv_model = Model(input=inp, output=out)
print(conv_model.summary())

(3000, 300)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 25, 300)      0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 25, 300, 1)   0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 25, 1, 300)   90300       lambda_3[0][0]                   
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 24, 1, 300)   180300      lambda_3[0][0]                   
_________________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [0]:
from keras.models import model_from_yaml
#add created model grapg in sequential model
model1 = Sequential()

model1.add(Embedding(vocabulary_size, embeddings_dim, input_length=max_sent_len, weights=[embedding_weights], trainable=False))
model1.add(Dropout(0.3))
model1.add(conv_model)# add model just like layer
model1.add(Flatten())
model1.add(Dropout(0.3))
model1.add(Dense(2048, activation='relu'))
model1.add(Dropout(0.3))
# model1.add(Dropout(0.5))
# model1.add(Dense(512, activation='relu'))
model1.add(Dense(train_labels.shape[1], activation='softmax'))


# Implement early stopping if accuracy doesn't increase for 10 epochs
# estopping = EarlyStopping(monitor='val_acc', patience=20)

# Save weights after each epoch if they're better than the previous ones
filepath = "weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 25, 300)           900000    
_________________________________________________________________
dropout_24 (Dropout)         (None, 25, 300)           0         
_________________________________________________________________
model_13 (Model)             (None, 1, 1, 1200)        991200    
_________________________________________________________________
flatten_7 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 1200)              0         
_________________________________________________________________
dense_35 (Dense)             (None, 2048)              2459648   
_________________________________________________________________
dropout_26 (Dropout)         (None, 2048)              0         
__________

In [0]:
optimizer = keras.optimizers.Adam(lr=LR,decay=1e-5)

model1.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [0]:
try:
    hist = model1.fit(train_sequences, train_labels,
                     validation_data=(test_sequences, test_labels),
                     epochs=500,
                     batch_size=8,
                     shuffle=False,
                     verbose=1,
                     callbacks=[checkpoint])
    
    score, acc = model1.evaluate(test_sequences,test_labels,
                                batch_size=16)

    print('Test score: ', score)
    print('Test accuracy: ', acc)
    model_yaml = model1.to_yaml()
    with open("model.yaml", "w") as yaml_file:
      yaml_file.write(model_yaml)
    model1.save_weights("predicthsc_model.h5")
    conv_model.save_weights("cnn_model.h5")
    print("save the model ")
    #model1.save('predicthsc_model.h5')
except KeyboardInterrupt:
    print('training stopped')

Train on 1417 samples, validate on 195 samples
Epoch 1/500
1417/1417 [==============================] - 5s 4ms/step - loss: 5.5081 - acc: 0.0621 - val_loss: 5.3468 - val_acc: 0.0718

Epoch 00001: val_acc improved from -inf to 0.07179, saving model to weights.best.hdf5
Epoch 2/500
1417/1417 [==============================] - 3s 2ms/step - loss: 5.1262 - acc: 0.0692 - val_loss: 5.0227 - val_acc: 0.1282

Epoch 00002: val_acc improved from 0.07179 to 0.12821, saving model to weights.best.hdf5
Epoch 3/500
1417/1417 [==============================] - 3s 2ms/step - loss: 4.7649 - acc: 0.0692 - val_loss: 4.5415 - val_acc: 0.1026

Epoch 00003: val_acc did not improve from 0.12821
Epoch 4/500
1417/1417 [==============================] - 3s 2ms/step - loss: 4.3874 - acc: 0.0819 - val_loss: 3.9856 - val_acc: 0.1436

Epoch 00004: val_acc improved from 0.12821 to 0.14359, saving model to weights.best.hdf5
Epoch 5/500
1417/1417 [==============================] - 3s 2ms/step - loss: 4.0519 - acc: 0.08

In [0]:
!ls

adc.json	     model.yaml			   weights.best.hdf5
cnn_model.h5	     predicthsc_model.h5	   weights.best.transfer.hdf5
encoder.pickle	     predicthsc_model_transfer.h5
model_transfer.yaml  sample_data


# Transfer learning using Keras

## Load pre-trained model and weights

In [0]:
#import model when using tf backend
import tensorflow as tf
yaml_file = open("model.yaml", 'r')
pretrained_model_yaml = yaml_file.read()
yaml_file.close()
pretrained_model = model_from_yaml(pretrained_model_yaml, custom_objects={"tf":tf})
pretrained_model.load_weights("weights.best.hdf5")

## Freeze the pretrained model and compile 

In [0]:
#make the model trainable state as False
for layer in pretrained_model.layers[:]:
    layer.trainable = False 
#evaluate, have to compile
pretrained_model.compile(loss='categorical_crossentropy',
               optimizer='adam',
              metrics=['accuracy'])
print(pretrained_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 25, 300)           900000    
_________________________________________________________________
dropout_24 (Dropout)         (None, 25, 300)           0         
_________________________________________________________________
model_13 (Model)             (None, 1, 1, 1200)        991200    
_________________________________________________________________
flatten_7 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 1200)              0         
_________________________________________________________________
dense_35 (Dense)             (None, 2048)              2459648   
_________________________________________________________________
dropout_26 (Dropout)         (None, 2048)              0         
__________

## Prove that the model is frozen

In [0]:
try:
    hist = pretrained_model.fit(train_sequences, train_labels,
                     validation_data=(test_sequences, test_labels),
                     epochs=10,
                     batch_size=8,
                     shuffle=False,
                     verbose=1,
                     callbacks=[checkpoint])
    
    score, acc = model1.evaluate(test_sequences,test_labels,
                                batch_size=16)

except KeyboardInterrupt:
    print('training stopped')

Train on 1417 samples, validate on 195 samples
Epoch 1/10
1417/1417 [==============================] - 4s 3ms/step - loss: 0.0108 - acc: 0.9979 - val_loss: 0.4096 - val_acc: 0.9487

Epoch 00001: val_acc did not improve from 0.94872
Epoch 2/10
1417/1417 [==============================] - 1s 874us/step - loss: 0.0055 - acc: 0.9972 - val_loss: 0.4096 - val_acc: 0.9487

Epoch 00002: val_acc did not improve from 0.94872
Epoch 3/10
1417/1417 [==============================] - 1s 867us/step - loss: 0.0080 - acc: 0.9972 - val_loss: 0.4096 - val_acc: 0.9487

Epoch 00003: val_acc did not improve from 0.94872
Epoch 4/10
1417/1417 [==============================] - 1s 867us/step - loss: 0.0084 - acc: 0.9979 - val_loss: 0.4096 - val_acc: 0.9487

Epoch 00004: val_acc did not improve from 0.94872
Epoch 5/10
1417/1417 [==============================] - 1s 862us/step - loss: 0.0093 - acc: 0.9958 - val_loss: 0.4096 - val_acc: 0.9487

Epoch 00005: val_acc did not improve from 0.94872
Epoch 6/10
1417/1417

In [0]:
for layer in pretrained_model.layers[:]:
    layer.trainable = False 

## Add custom Layers

In [0]:
x = pretrained_model.layers[-2].output
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(train_labels.shape[1], activation="softmax")(x)

## Create a new transfer model

In [0]:
model_transfer = Model(inputs = pretrained_model.input, outputs = predictions)

# Define new checkpoint
# Save weights after each epoch if they're better than the previous ones
filepath = "weights.best.transfer.hdf5"
new_checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

#compile the model
model_transfer.compile(loss='categorical_crossentropy',
               optimizer='adam',
              metrics=['accuracy'])
print(model_transfer.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6_input (InputLaye (None, 25)                0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 25, 300)           900000    
_________________________________________________________________
dropout_24 (Dropout)         (None, 25, 300)           0         
_________________________________________________________________
model_13 (Model)             (None, 1, 1, 1200)        991200    
_________________________________________________________________
flatten_7 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 1200)              0         
_________________________________________________________________
dense_35 (Dense)             (None, 2048)              2459648   
__________

## Train the new model

In [0]:
try:
    hist = model_transfer.fit(train_sequences, train_labels,
                     validation_data=(test_sequences, test_labels),
                     epochs=500,
                     batch_size=8,
                     shuffle=False,
                     verbose=1,
                     callbacks=[new_checkpoint])
    
    score, acc = model_transfer.evaluate(test_sequences,test_labels,
                                batch_size=16)

    print('Test score: ', score)
    print('Test accuracy: ', acc)
    transfermodel_yaml = model_transfer.to_yaml()
    with open("model_transfer.yaml", "w") as yaml_file:
      yaml_file.write(transfermodel_yaml)
    #Save the final weights
    model_transfer.save_weights("predicthsc_model_transfer.h5")
    print("save the model ")

except KeyboardInterrupt:
    print('training stopped')

Train on 1417 samples, validate on 195 samples
Epoch 1/500
1417/1417 [==============================] - 6s 4ms/step - loss: 6.8554 - acc: 0.0635 - val_loss: 5.3696 - val_acc: 0.0821

Epoch 00001: val_acc improved from -inf to 0.08205, saving model to weights.best.transfer.hdf5
Epoch 2/500
1417/1417 [==============================] - 2s 1ms/step - loss: 5.1983 - acc: 0.0289 - val_loss: 5.0808 - val_acc: 0.1282

Epoch 00002: val_acc improved from 0.08205 to 0.12821, saving model to weights.best.transfer.hdf5
Epoch 3/500
1417/1417 [==============================] - 2s 1ms/step - loss: 4.8175 - acc: 0.1059 - val_loss: 4.2799 - val_acc: 0.1436

Epoch 00003: val_acc improved from 0.12821 to 0.14359, saving model to weights.best.transfer.hdf5
Epoch 4/500
1417/1417 [==============================] - 2s 1ms/step - loss: 4.1243 - acc: 0.1440 - val_loss: 3.5330 - val_acc: 0.2974

Epoch 00004: val_acc improved from 0.14359 to 0.29744, saving model to weights.best.transfer.hdf5
Epoch 5/500
1417/141

In [0]:
!ls

adc.json	     model.yaml			   weights.best.hdf5
cnn_model.h5	     predicthsc_model.h5	   weights.best.transfer.hdf5
encoder.pickle	     predicthsc_model_transfer.h5
model_transfer.yaml  sample_data


In [0]:
#add data into cells
def w2sheet(data, cells):
  for i in range(len(data)):
    cells[i].value = data[i]
  return cells

In [0]:
prediction = model1.predict(test_sequences, batch_size=32)
print(prediction)

[[3.8168166e-09 8.7680533e-08 5.6960804e-14 ... 6.6239306e-15
  9.8743721e-15 9.9999917e-01]
 [9.9996114e-01 1.1575045e-07 2.0985910e-10 ... 3.7521948e-23
  9.2123731e-27 7.2596039e-16]
 [9.9993944e-01 1.0576472e-06 6.8997412e-09 ... 2.2447872e-20
  1.4652124e-23 2.5653584e-13]
 ...
 [2.7811712e-08 4.9291491e-12 3.5978546e-15 ... 6.7690336e-27
  1.4337554e-31 2.9645158e-33]
 [3.1291576e-09 1.4302677e-10 5.1887339e-09 ... 2.1104136e-11
  1.3210497e-13 1.2392986e-13]
 [9.1330481e-01 9.3762793e-08 2.0844550e-12 ... 9.1484744e-28
  2.8512224e-34 2.8996194e-24]]


In [0]:
!ls

adc.json	     model.yaml			   weights.best.hdf5
cnn_model.h5	     predicthsc_model.h5	   weights.best.transfer.hdf5
encoder.pickle	     predicthsc_model_transfer.h5
model_transfer.yaml  sample_data


In [0]:
# import pickle
# with open('encoder.pickle', 'rb') as handle:
#   encoder = pickle.load(handle)
#   tokenizer1 = encoder['tokenizer']
  
#   test_query = sequence.pad_sequences(tokenizer1.texts_to_sequences([filter_string('PUMP, FUEL OIL SERVICE')]) , maxlen=15)
#   prediction = loaded_model.predict(test_query)
#   classificaitons = prediction[0].tolist()
#   result_list = list(zip(encoder['classes_'],classificaitons))
#   result_list.sort(key=lambda x: x[1], reverse=True)
#   print(result_list)

#Saving

In [0]:
with open("model.yaml", "w") as yaml_file:
      yaml_file.write(model_yaml)
files.download('model.yaml')
#import pickle
#from google.colab import files

#encoder = {"tokenizer":tokenizer, "classes_": lb.classes_}
#print(encoder["classes_"].shape)
#with open('encoder.pickle', 'wb') as file:
#  pickle.dump(encoder,file)
#files.download('encoder.pickle')
#files.download('weights.best.hdf5')

In [0]:
uploaded = drive.CreateFile({'title': 'weights.best.hdf5'})
uploaded.SetContentFile('weights.best.hdf5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1T7D5gMlCOOtv3smlZHZ2Nb5TbSIeeOMi


# Error Analysis

In [0]:
#function for making error analysis
import tensorflow as tf
import random
#print(test_texts)
prediction = model1.predict(test_sequences, batch_size=32)
print(prediction.shape)
#print("test_x:",test_labels)
#order of classes
#print("Classes that are considered in the problem : {0}".format(repr( lb.classes_ ))) 
#print(lb.classes_.shape)
#print("test_y.shape:",test_labels.shape)
#print("predic_class:", np.max(prediction,axis=1))
#print("prediction.shape",prediction.shape)
prediction_top_1 = tf.nn.top_k(tf.convert_to_tensor(prediction, np.float32), 1)
prediction_top_1_index = tf.argmax(prediction, 1)
prediction_top_5 = tf.nn.top_k(tf.convert_to_tensor(prediction, np.float32), 5)
sh = gc.create("Error2")
worksheet = gc.open("Error2").sheet1
#set cell name and range
worksheet.update_acell('A1', "test_texts")
worksheet.update_acell('B1', "predict_prob_top1")
worksheet.update_acell('C1', "predict_class_top1")
worksheet.update_acell('D1', "target")
worksheet.update_acell('E1', "predict_prob_top5")
worksheet.update_acell('F1',"predict_class_top5")
cells_text = worksheet.range('A2:A272')
cells_predict_probtop1 = worksheet.range('B2:B272')
worksheet.update_cells(w2sheet(test_texts, cells_text))
cells_predict_classtop1 = worksheet.range('C2:C272')
cells_target = worksheet.range('D2:D272')
cells_predict_probtop5 = worksheet.range('E2:E272')
cells_predict_classtop5 = worksheet.range('F2:F272')
worksheet.update_cells(w2sheet(lb.inverse_transform(test_labels), cells_target))
#flattended_text = test_texts
# for i in range(len(test_texts)):
#   cells_text[i].value = test_texts[i]
# worksheet.update_cells(cells_text)
with tf.Session() as sess:
  #print("probability fro top1:", sess.run(prediction_top_1))
  #print("prediction_top_1_index:", sess.run(prediction_top_1_index))
  target_index = sess.run(prediction_top_1_index)
  predict_classes1 = []
  for i in target_index:
      predict_class = lb.classes_[i]
      predict_classes1.append(predict_class)
  #print(predict_classes)
  #print("probability for top5:", sess.run(prediction_top_5)[0])
  worksheet.update_cells(w2sheet(sess.run(prediction_top_1)[0].flatten(),cells_predict_probtop1))
  worksheet.update_cells(w2sheet(predict_classes1, cells_predict_classtop1))
  worksheet.update_cells(w2sheet(sess.run(prediction_top_5)[0],cells_predict_probtop5))
  target_index = sess.run(prediction_top_5)[1]
  predict_classes5 = []
  for i in range(target_index.shape[0]):
    predict_class = lb.classes_[target_index[i]]
    predict_classes5.append(predict_class)
  worksheet.update_cells(w2sheet(predict_classes5, cells_predict_classtop5))
      

(195, 240)


In [0]:
worksheet.update_acell('G1',"accuracy count")
cells_acc = worksheet.range('G2:G272')
count = np.zeros((1,test_labels.shape[0]))
for i in range(test_labels.shape[0]):
  if cells_target[i].value == cells_predict_classtop1[i].value:
    count[0][i] = 1
worksheet.update_cells(w2sheet(count.flatten(), cells_acc))


In [0]:
#function for making error analysis
import tensorflow as tf
import random
#print(test_texts)
prediction = model1.predict(train_sequences, batch_size=32)
print(prediction.shape)
#print("test_x:",test_labels)
#order of classes
#print("Classes that are considered in the problem : {0}".format(repr( lb.classes_ ))) 
#print(lb.classes_.shape)
#print("test_y.shape:",test_labels.shape)
#print("predic_class:", np.max(prediction,axis=1))
#print("prediction.shape",prediction.shape)
prediction_top_1 = tf.nn.top_k(tf.convert_to_tensor(prediction, np.float32), 1)
prediction_top_1_index = tf.argmax(prediction, 1)
prediction_top_5 = tf.nn.top_k(tf.convert_to_tensor(prediction, np.float32), 5)
#sh = gc.create("Error3")
worksheet = gc.open("Error2").sheet1
#set cell name and range
worksheet.update_acell('A1', "train_texts")
worksheet.update_acell('B1', "predict_prob_top1")
worksheet.update_acell('C1', "predict_class_top1")
worksheet.update_acell('D1', "target")
worksheet.update_acell('E1', "predict_prob_top5")
worksheet.update_acell('F1',"predict_class_top5")
cells_text = worksheet.range('A2:A1550')
cells_predict_probtop1 = worksheet.range('B2:B1550')
worksheet.update_cells(w2sheet(train_texts, cells_text))
cells_predict_classtop1 = worksheet.range('C2:C1550')
cells_target = worksheet.range('D2:D1550')
cells_predict_probtop5 = worksheet.range('E2:E1550')
cells_predict_classtop5 = worksheet.range('F2:F1550')
worksheet.update_cells(w2sheet(lb.inverse_transform(train_labels), cells_target))
#flattended_text = test_texts
# for i in range(len(test_texts)):
#   cells_text[i].value = test_texts[i]
# worksheet.update_cells(cells_text)
with tf.Session() as sess:
  #print("probability fro top1:", sess.run(prediction_top_1))
  #print("prediction_top_1_index:", sess.run(prediction_top_1_index))
  target_index = sess.run(prediction_top_1_index)
  predict_classes1 = []
  for i in target_index:
      predict_class = lb.classes_[i]
      predict_classes1.append(predict_class)
  #print(predict_classes)
  #print("probability for top5:", sess.run(prediction_top_5)[0])
  worksheet.update_cells(w2sheet(sess.run(prediction_top_1)[0].flatten(),cells_predict_probtop1))
  worksheet.update_cells(w2sheet(predict_classes1, cells_predict_classtop1))
  worksheet.update_cells(w2sheet(sess.run(prediction_top_5)[0],cells_predict_probtop5))
  target_index = sess.run(prediction_top_5)[1]
  predict_classes5 = []
  for i in range(target_index.shape[0]):
    predict_class = lb.classes_[target_index[i]]
    predict_classes5.append(predict_class)
  worksheet.update_cells(w2sheet(predict_classes5, cells_predict_classtop5))
worksheet.update_acell('G1',"accuracy count")
cells_acc = worksheet.range('G2:G272')
count = np.zeros((1,test_labels.shape[0]))
for i in range(test_labels.shape[0]):
  if cells_target[i].value == cells_predict_classtop1[i].value:
    count[0][i] = 1
worksheet.update_cells(w2sheet(count.flatten(), cells_acc))

      

(1417, 240)


RequestError: ignored

In [0]:
#output predicted class and prob
import tensorflow as tf
prediction = model1.predict(test_sequences, batch_size=32)
print(prediction.shape)
prediction_top_10 = tf.nn.top_k(tf.convert_to_tensor(prediction, np.float32), 1)
with tf.Session() as sess:
  target_index = sess.run(prediction_top_10)[1]
  top10_prob = sess.run(prediction_top_10)[0]
  print("top10 prob:", top10_prob)
  predict_classes_top10 = []
  for i in target_index:
      predict_class = lb.classes_[i]
      predict_classes_top10.append(predict_class)
  print(predict_classes_top10)

In [0]:
#test
# sh = gc.create("Error Analysis")
# worksheet = gc.open("Error Analysis").sheet1
# cell_list = worksheet.range('A1:A3')

# import random
# for cell in cell_list:
#   cell.value = random.randint(1, 10)
# worksheet.update_cells(cell_list)

In [0]:
print( os.getcwd() )
print( os.listdir() )
#download function
from google.colab import files
#model
files.download( "model.yaml" ) 
#parameters
files.download( "predicthsc_model.h5" ) 

In [0]:
!ls


In [0]:
from keras.models import model_from_yaml
import tensorflow as tf
yaml_file = open("model.yaml", 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml, {'tf': tf})
loaded_model.load_weights("predicthsc_model.h5")
print("loaded model.....")

#evaluate
loaded_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(loaded_model.summary())